In [7]:
mse <- function(y, yhat) mean((y - yhat)^2)
#install.packages(c('glmnet','gglasso','genlasso','pls','Matrix','imager'))
# -------------------- LASSO via glmnet -----------------------------
# Uses glmnet and cv.glmnet to select lambda
lasso_glmnet <- function(X, y, nfolds = 5, alpha = 1) {
  if (!requireNamespace('glmnet', quietly = TRUE)) stop('Install package glmnet')
  library(glmnet)
  # glmnet expects x as matrix, y as vector
  cvfit <- cv.glmnet(x = X, y = y, alpha = alpha, nfolds = nfolds, standardize = TRUE)
  lambda_min <- cvfit$lambda.min
  coef_vec <- as.numeric(coef(cvfit, s = "lambda.min")) # includes intercept at index 1
  intercept <- coef_vec[1]
  beta <- coef_vec[-1]
  list(cvfit = cvfit, lambda_min = lambda_min, intercept = intercept, beta = beta,summary=summary(cvfit))
}

predict_lasso_glmnet <- function(fit, Xnew) {
  predict(fit$cvfit, newx = Xnew, s = "lambda.min")
}
# Generating data
Data_generation <- function(n = 200, p = 20, s = 7, sigma = 1, seed = 1) {
  set.seed(seed)
  # Sets the seed for random number generation. This ensures that if you run
  # the function with the same seed, you will get the exact same simulated data
  # each time.

  X <- matrix(rnorm(n * p), n, p)

  beta <- rep(0, p) #true beta

  beta[1:s] <- seq(from = 2, length.out = s, by = -0.3)
  # This means only the first 's' predictors in X will have a non-zero effect on y.

  y <- X %*% beta + rnorm(n, sd = sigma)
  # The error is added to the linear combination to get the final response 'y'.

  list(X = X, y = as.numeric(y), beta = beta)
  # Returns a list containing the generated data and the true coefficients:
  # X: the predictor matrix
  # y: the response vector (converted to a numeric vector using as.numeric)
  # beta: the true coefficient vector
}
L=Data_generation(n=50)
Scaled_X=scale(L$X, center = TRUE, scale = TRUE)
model_small=lasso_glmnet(X=Scaled_X,y=L$y)
mse_small=mse(L$y,predict_lasso_glmnet(model_small,Scaled_X))
L=L=Data_generation(n=250)
Scaled_X=scale(L$X, center = TRUE, scale = TRUE)
model_moderate=lasso_glmnet(X=Scaled_X,y=L$y)
mse_moderate=mse(L$y,predict_lasso_glmnet(model_moderate,Scaled_X))
L=Data_generation(n=1000)
Scaled_X=scale(L$X, center = TRUE, scale = TRUE)
model_large=lasso_glmnet(X=Scaled_X,y=L$y)
mse_large=mse(L$y,predict_lasso_glmnet(model_large,Scaled_X))

In [ ]:
# -------------------- Group LASSO via gglasso ----------------------
# groups: integer vector of length p assigning each variable to a group
group_lasso_gglasso <- function(X, y, groups, nfolds = 5) {
  if (!requireNamespace('gglasso', quietly = TRUE)) stop('Install package gglasso')
  library(gglasso)
  stopifnot(length(groups) == ncol(X))
  # gglasso wants X, y, group, loss = "ls"
  cvfit <- cv.gglasso(x = X, y = y, group = groups, loss = "ls", nfolds = nfolds)
  lambda_min <- cvfit$lambda.min
  beta_full <- coef(cvfit, s = "lambda.min")
  intercept <- as.numeric(beta_full[1])
  beta <- as.numeric(beta_full[-1])
  list(cvfit = cvfit, lambda_min = lambda_min, intercept = intercept, beta = beta)
}

predict_group_gglasso <- function(fit, Xnew) {
  predict(fit$cvfit, newx = Xnew, s = "lambda.min")
}
